# Create Admission List

This script is used for creating a list of admission ids. We only keep admissions that are the first admissions of their patients.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math

from utils import getConnection

%matplotlib inline

In [2]:
try:
    conn = getConnection()
    print('Connected to Postgre Database!')
except:
    print('Fail to connect!')

Connected to Postgre Database!


## Select all admissions

We collect all admission_ids from TABLE ICUSTAYS and TABLE TRANSFERS

In [3]:
# Select from icustay and transfer table
cur = conn.cursor()
cur.execute('create table if not exists admission_ids as (select distinct hadm_id from mimiciii.icustays union select distinct hadm_id from mimiciii.transfers)')
conn.commit()

cur = conn.cursor()
cur.execute('select * from admission_ids')
res = cur.fetchall()

admission_ids = [r[0] for r in res]
admission_ids_txt = ','.join(map(str, admission_ids))

In [4]:
# number of admission id
print('len(admission_ids) = ', len(admission_ids))

len(admission_ids) =  58976


In [5]:
# save to admission_ids.npy
tosave = {'admission_ids':admission_ids, 'admission_ids_txt': admission_ids_txt}
np.save('res/admission_ids.npy',tosave)

Make sure that there is no duplication in admission_ids.

In [6]:
print(len(admission_ids), len(set(admission_ids)))

58976 58976


## Remove non-first admissions

We remove all admissions which are not the first admissions of some patients in order to prevent possible information leakage, which will happen when multiple admissions of the same patient occur in training set and test set simultaneously.

In [2]:
# get the list of admission ids which is the first admission of the subject
conn = getConnection()
cur = conn.cursor()
cur.execute('select hadm_id from admission_ids where hadm_id in (select distinct on (subject_id) hadm_id from (select * from admissions order by admittime) tt)')
res = cur.fetchall()

admission_first_ids = [r[0] for r in res]
admission_first_ids_txt = ','.join(list(map(str, admission_first_ids)))
tosave = {'admission_ids': admission_first_ids, 'admission_ids_txt': admission_first_ids_txt}
np.save('res/admission_first_ids.npy', tosave)
print(len(admission_first_ids))

46520
